In [129]:
from PIL import Image
from torchvision import transforms
import numpy as np
import os
import json
import torch
import random
import shutil
from collections import defaultdict
import cv2

In [130]:
# Đường dẫn thư mục đầu ra
out_path = r"D:\Downloads\PyTorch\tt100kv1\images"
os.makedirs(out_path, exist_ok=True)

# Đường dẫn đến thư mục chứa ảnh và file annotation
folder_path = r"D:\Downloads\PyTorch\tt100k_2021\tt100k_2021\test"
annotation_path = r"D:\Downloads\PyTorch\tt100kv1\annotations_all.json"
# out_data_val = os.path.join(out_path, "val")
out_data_train = os.path.join(out_path, "train")

# Tạo thư mục output nếu chưa có
# os.makedirs(out_data_val, exist_ok=True)
os.makedirs(out_data_train, exist_ok=True)
os.makedirs(os.path.join(out_path, "test"), exist_ok=True)
os.makedirs(os.path.join(out_path, "img_not_anno"), exist_ok=True)
os.makedirs(os.path.join(out_path, "other"), exist_ok=True)
# Danh sách các frame cần xóa và danh sách frame validation
delete_frame = []
frame_valid = []
frame_not_anno = []

In [131]:
def preprocessing_image(folder_path, annotation_path):
    # Mở file annotation
    with open(annotation_path, "r") as f:
        annotations = json.load(f)
    count = 0
    # Duyệt folder data ảnh
    for filename in os.listdir(folder_path):
        
        # Kiểm tra file có đúng định dạng cần
        if filename.endswith((".jpg", ".png", ".jpeg")):
            
            # Lấy đường dẫn file 
            img_path = os.path.join(folder_path, filename)
            relative_path = os.path.join(os.path.basename(os.path.dirname(img_path)), 
                                         os.path.basename(img_path))
            img_id = os.path.splitext(filename)[0]
            
            # Kiểm tra id ảnh trong folder có trong annotation không
            if img_id in annotations['imgs']:
                count += 1
                anno_img_path = annotations['imgs'][img_id]['path']
                
                # Kiểm tra đường dẫn trong folder và annotation có giống nhau không
                if os.path.dirname(relative_path) != anno_img_path.split('/')[0]:
                    print(f"{relative_path} and {anno_img_path}")
                    
                    # delete_frame.append(img_id)
                    # continue
                    
            # Thêm vào danh sách frame cần xóa trong annotation
            else:
                # delete_frame.append(img_id)
                # print("id không có ", img_id)
                frame_not_anno.append(img_id)
                with Image.open(img_path) as img:
                # Chuyển đổi ảnh thành dạng RGB để làm model faster R-CNN
                    if img.mode != "RGB":
                        print(filename + " không cùng RGB")
                        img = img.convert("RGB")
                out_data = os.path.join(out_path, 'img_not_anno', os.path.basename(relative_path))
                os.makedirs(os.path.dirname(out_data), exist_ok=True)
                # img.save(out_data, quality=100)
                shutil.copy2(img_path, out_data)
                continue
                
            with Image.open(img_path) as img:
                # Chuyển đổi ảnh thành dạng RGB để làm model faster R-CNN
                if img.mode != "RGB":
                    print(filename + " không cùng RGB")
                    img = img.convert("RGB")
                # Lưu ảnh hợp lệ vào folder
            out_data = os.path.join(out_path, relative_path)
            os.makedirs(os.path.dirname(out_data), exist_ok=True)
                # img.save(out_data, quality=100)
            shutil.copy2(img_path, out_data)
        else:
            print(filename + " không đúng định dạng")
    
    print("Success ", count)
    print("Frame not anno ", frame_not_anno)
    print("Size frame not anno ", len(frame_not_anno))
def modify_annotation(delete_frame, annotation_path, out_path):
    # Mở file annotation
    with open(annotation_path, "r") as f:
        annotations = json.load(f)
    
    # Duyệt qua tất cả các ảnh trong annotations
    for image_id in list(annotations["imgs"].keys()):
        # Đặt cờ kiểm tra tính hợp lệ của bounding box
        valid_bbox = True
        # Cập nhật từng bounding box trong ảnh
        for annotation in annotations["imgs"][image_id]["objects"]:
            bbox = annotation.get("bbox", {})
            if annotation["category"].find("pa") != -1:
                # print(category)
                annotation["category"] = "pa*"
            elif annotation["category"].find("pl") != -1:
                # print(category)
                annotation["category"] = "pl*"
            elif annotation["category"].find("pm") != -1:
                # print(category)
                annotation["category"] = "pm*"
            elif annotation["category"].find("pr") != -1:
                # print(category)
                annotation["category"] = "pr*"
            elif annotation["category"].find("pw") != -1:
                # print(category)
                annotation["category"] = "pw*"
            elif annotation["category"].find("ph") != -1:
                # print(category)
                annotation["category"] = "ph*"
            elif annotation["category"].find("il") != -1:
                # print(category)
                annotation["category"] = "il*"
            if all(key in bbox for key in ["xmin", "ymin", "xmax", "ymax"]):
                try:
                    # Lấy các giá trị xmin, ymin, xmax, ymax
                    xmin = float(bbox["xmin"])
                    ymin = float(bbox["ymin"])
                    xmax = float(bbox["xmax"])
                    ymax = float(bbox["ymax"])

                except (ValueError, TypeError) as e:
                    valid_bbox = False
                    print(f"Lỗi ở bounding box trong ảnh {image_id}: {e}")
                    break
            else:
                valid_bbox = False
                print(f"Không đủ tham số {image_id}")
                break
        
        # Nếu bounding box không hợp lệ, thêm vào delete_frame
        if not valid_bbox:
            delete_frame.append(image_id)
    
    # In ra delete_frame để kiểm tra xem các ảnh nào đã bị xóa
    print("Các ảnh bị xóa:", delete_frame)
    print(len(delete_frame))
    # Loại bỏ các ảnh trong delete_frame khỏi annotations
    # for image_id in delete_frame:
    #     if image_id in annotations["imgs"]:
    #         del annotations["imgs"][image_id]
            
    # Lưu lại file annotation đã cập nhật
    modified_annotation_path = os.path.join(out_path, "annotations_all.json")
    with open(modified_annotation_path, "w") as file:
        json.dump(annotations, file, indent=4)

    print("File annotation đã được cập nhật và lưu lại.")

In [132]:
# # Gọi các hàm tiền xử lý
# preprocessing_image(folder_path, annotation_path)
# folder_path = r"D:\Downloads\PyTorch\tt100k_2021\tt100k_2021\train"
# preprocessing_image(folder_path, annotation_path)
# folder_path = r"D:\Downloads\PyTorch\tt100k_2021\tt100k_2021\other"
# preprocessing_image(folder_path, annotation_path)

In [133]:

# modify_annotation(delete_frame,annotation_path, out_path)
# print(frame_not_anno)
# print(len(frame_not_anno))

In [134]:
# annotation_path = r"D:\Downloads\PyTorch\tt100kv1\annotations_all.json"
delete_frame = []
test_count = {}
train_count = {}
other_count = {}
rare_classes = {}
popular_classes = {}
delete_frame_only = []
class_in_frame_in_rare_test = []
class_in_frame_in_rare_train = []
class_in_frame_in_rare_other = []
exists_categories = {}
folder_path = r"D:\Downloads\PyTorch\tt100kv1\images"
out_path = r"D:\Downloads\PyTorch\tt100kv1"
os.makedirs(out_path, exist_ok=True)
def compare_dicts(dict1, dict2):
    # Duyệt qua các key trong dict1
    only_dcit1 = []
    rare_dict2 = []
    for key in dict1:
        if key not in dict2:
            only_dcit1.append(key)
            print(f"Key '{key}' chỉ có trong dict1 với giá trị: {dict1[key]}")
    # Duyệt qua các key trong dict2 để tìm các key không có trong dict1
    # for key in dict2:
    #     if key not in dict1:
    #         print(f"Key '{key}' chỉ có trong dict2 với giá trị: {dict2[key]}")
    for key in dict1:
        if key in dict2:
            if dict2[key] < 11:
                rare_dict2.append(key)
                print(f"Giá trị của key '{key}' khác nhau: {dict1[key]} và {dict2[key]}")
    return only_dcit1, rare_dict2
def ThongKeAnnotation(annotation_path, test_count,train_count,other_count,rare_class={}, popular_class={}, exists_categories={}):   
    with open(annotation_path, "r") as f:
        annotations = json.load(f)
    print(len(annotations["imgs"]))
    # thống kê các class tồn tại và tần suất xuất hiện trong annotations 
    dirname_data = ['train','val','test']
    for img_id , img_info in annotations["imgs"].items():
        dirname = img_info['path'].split('/')[0]
        # if dirname != 'other':
        for obj in img_info["objects"]:
            if obj["category"] in exists_categories:
                exists_categories[obj["category"]] += 1
            else: exists_categories[obj["category"]] = 1
            if dirname == 'test':
                if obj["category"] in test_count: test_count[obj["category"]] += 1
                else: test_count[obj["category"]] = 1
            elif dirname == 'train':
                if obj["category"] in train_count: train_count[obj["category"]] += 1
                else: train_count[obj["category"]] = 1
            elif dirname == 'other':
                if obj["category"] in other_count: other_count[obj["category"]] += 1
                else: other_count[obj["category"]] = 1
    # tìm những class xuất hiện ít và nhiều
    for key, value in exists_categories.items():
        if value < 51: 
            if key in rare_class: rare_class[key] += value
            else: rare_class[key] = value
        elif value > 1000: 
            if key in popular_class: popular_class[key] += value
            else: popular_class[key] = value
    #in kết quả
    exists_categories = dict(sorted(exists_categories.items(), key=lambda item: item[1]))
    print(exists_categories)
    print(len(exists_categories.keys()))
    print(exists_categories.values())
    test_count = dict(sorted(test_count.items(), key=lambda item: item[1]))
    train_count = dict(sorted(train_count.items(), key=lambda item: item[1]))
    other_count = dict(sorted(other_count.items(), key=lambda item: item[1]))
    print()
    print("test: length",len(test_count),test_count)
    print("test",test_count.values())
    print()
    print("train: ",len(train_count),train_count)
    print("train",train_count.values())
    print()
    print("other: ",len(other_count),other_count)
    print("other",other_count.values())
def CheckFrameWithClasses(annotation_path, test_missing_classes, train_missing_classes, other_missing_classes, rare_class, popular_class, delete_frame_only, class_in_frame_in_rare_test=[],class_in_frame_in_rare_train=[],class_in_frame_in_rare_other=[]):    
    with open(annotation_path, "r") as f:
        annotations = json.load(f)
    frames_only_test = []
    frames_only_other = []
    frames_only_train = []
    frames_only_test_and_other = []
    frames_in_rare_test = []
    frames_in_rare_train = []
    frames_in_rare_other = []
    frames_in_rare_and_popular_test = []
    frames_in_rare_and_popular_train = []
    frames_in_rare_and_popular_other = []   
    class_in_frame_in_rare_popular_test = []
    class_in_frame_in_rare_popular_train = []
    class_in_frame_in_rare_popular_other = []
    dirname_data = ['train','other','test']
    for img_id, img_info in annotations["imgs"].items():
        check = False
        check_rare = False
        check_popular = False  
        check_frame_only_test = False
        check_frame_only_other = False
        dirname = img_info['path'].split('/')[0]
        # Duyệt tìm frame chỉ có trong 1 tập #frames_only_test cần test_missing_classes
        for obj in img_info["objects"]:
            if dirname in dirname_data:
                if obj['category'] in test_missing_classes:
                    frames_only_test.append(img_id)
                    break
                elif obj['category'] in train_missing_classes:
                    frames_only_train.append(img_id)
                    break
                elif obj['category'] in other_missing_classes:
                    frames_only_other.append(img_id)
                    break
            
        if dirname in dirname_data:
            
            # Duyệt tìm frame có class có trong tập test và other
            if dirname != 'train':
                for obj in img_info["objects"]:
                    if obj["category"] in test_missing_classes:
                        check_frame_only_test = True
                        if check_frame_only_other == True: break
                    elif obj["category"] in other_missing_classes:
                        check_frame_only_other = True
                        if check_frame_only_test == True: break
                if check_frame_only_test == True and check_frame_only_other == True:        
                    frames_only_test_and_other.append(img_id)
                    
            # Duyệt tìm frame có class hiếm và phổ biến trong tập test or train or other
            for obj in img_info["objects"]:
                if obj["category"] in rare_class:
                    check_rare = True 
                    if check_popular: 
                        break
                elif obj["category"] in popular_class:
                    check_popular = True
                    if check_rare: break    
            if check_rare == True and check_popular == True: 
                if dirname == 'test':
                    frames_in_rare_and_popular_test.append(img_id)
                elif dirname == 'train':
                    frames_in_rare_and_popular_train.append(img_id)
                elif dirname == 'other':
                    frames_in_rare_and_popular_other.append(img_id)

            # Duyệt tìm frame có class hiếm trong tập test or train or other
            for obj in img_info["objects"]:
                if obj["category"] in rare_class:
                    check = True   
                elif obj["category"] in popular_class:
                    check = False
                    break
            if check == True: 
                if dirname == 'test':
                    frames_in_rare_test.append(img_id)
                elif dirname == 'train':
                    frames_in_rare_train.append(img_id)
                elif dirname == 'other':
                    frames_in_rare_other.append(img_id)
                    
    class_in_delete_frame_only = []
    for img_id, img_info in annotations["imgs"].items():
        if img_id in delete_frame_only:
            class_in_delete_frame_only.append(img_id)
        else:
            if img_id in frames_in_rare_test:
                class_in_frame_in_rare_test.append(img_id)
            elif img_id in frames_in_rare_train:
                class_in_frame_in_rare_train.append(img_id)
            elif img_id in frames_in_rare_other:
                class_in_frame_in_rare_other.append(img_id)
            elif img_id in frames_in_rare_and_popular_test:
                class_in_frame_in_rare_popular_test.append(img_id)
            elif img_id in frames_in_rare_and_popular_train:
                class_in_frame_in_rare_popular_train.append(img_id)
            elif img_id in frames_in_rare_and_popular_other:
                class_in_frame_in_rare_popular_other.append(img_id)
    print("Frame có class chỉ có trong test ", frames_only_test)
    print("số lượng ", len(frames_only_test))
    print()
    print("Frame có class chỉ có trong train ", frames_only_train)
    print("số lượng ", len(frames_only_train))
    print()
    print("Frame có class chỉ có trong other ", frames_only_other)
    print("số lượng ", len(frames_only_other))
    print()
    print("Frame có class chỉ có trong test và other ", frames_only_test_and_other)
    print("số lượng ", len(frames_only_test_and_other))
    print()
    print("Frame có class hiếm trong tập test ", class_in_frame_in_rare_test)
    print("số lượng ", len(class_in_frame_in_rare_test))
    print()
    print("Frame có class hiếm trong tập train ", class_in_frame_in_rare_train)
    print("số lượng ", len(class_in_frame_in_rare_train))
    print()
    print("Frame có class hiếm trong tập other ", class_in_frame_in_rare_other)
    print("số lượng ", len(class_in_frame_in_rare_other))
    print()
    print("Frame có class hiếm và phổ biến trong tập test ", class_in_frame_in_rare_popular_test)
    print("số lượng ", len(class_in_frame_in_rare_popular_test))
    print()
    print("Frame có class hiếm và phổ biến trong tập train ", class_in_frame_in_rare_popular_train)
    print("số lượng ", len(class_in_frame_in_rare_popular_train))
    print()
    print("Frame có class hiếm và phổ biến trong tập other ", class_in_frame_in_rare_popular_other)
    print("số lượng ", len(class_in_frame_in_rare_popular_other))
    # return delete_frame

In [135]:
ThongKeAnnotation(annotation_path,test_count,train_count,other_count,rare_classes, popular_classes, exists_categories)

10592
{'w28': 1, 'w48': 1, 'w5': 1, 'w49': 1, 'w44': 1, 'w1': 1, 'w62': 1, 'p7': 1, 'w14': 1, 'pclr': 1, 'w56': 1, 'w16': 2, 'p20': 2, 'w12': 2, 'p21': 2, 'pcs': 2, 'p24': 2, 'pnlc': 2, 'w38': 3, 'w31': 3, 'w60': 3, 'w50': 3, 'p28': 3, 'w15': 3, 'w35': 3, 'i11': 3, 'w37': 4, 'w66': 4, 'p4': 4, 'i14': 5, 'pn-2': 5, 'w10': 5, 'p15': 5, 'i15': 6, 'pctl': 6, 'pc': 7, 'w8': 7, 'pt': 7, 'i3': 7, 'i1': 8, 'w43': 8, 'w46': 8, 'w41': 9, 'p1n': 12, 'w18': 13, 'w26': 13, 'p8': 13, 'w20': 15, 'w24': 16, 'p2': 16, 'i13': 17, 'pcd': 18, 'w34': 18, 'pcr': 19, 'p16': 21, 'pss': 22, 'w3': 22, 'i12': 23, 'iz': 24, 'w42': 27, 'w45': 32, 'p17': 33, 'p29': 33, 'pw*': 35, 'wc': 39, 'pdd': 43, 'w47': 43, 'p14': 49, 'p25': 50, 'w21': 57, 'w22': 64, 'p9': 65, 'im': 70, 'i10': 71, 'pbm': 72, 'p1': 74, 'p18': 75, 'pb': 76, 'ps': 77, 'pcl': 86, 'w63': 95, 'w58': 95, 'pbp': 96, 'w30': 97, 'pa*': 105, 'p6': 116, 'w32': 124, 'p19': 129, 'w13': 130, 'p27': 135, 'pg': 157, 'p3': 173, 'w55': 179, 'p12': 189, 'w59': 231

In [136]:
test_missing_classes = {
    cls: values for cls, values in test_count.items() if cls not in train_count and cls not in other_count
}
print("Classes only in test:", test_missing_classes)
print("Values: ", test_missing_classes.values())
train_missing_classes = {
    cls: values for cls, values in train_count.items() if cls not in test_count and cls not in other_count
}
print("Classes only in train:", train_missing_classes)
print("Values: ", train_missing_classes.values())
other_missing_classes = {
    cls: values for cls, values in other_count.items() if cls not in test_count and cls not in train_count
}
print("Classes only in other:", other_missing_classes)
print("other: ", other_missing_classes.values())
CheckFrameWithClasses(annotation_path, test_missing_classes, train_missing_classes, other_missing_classes, rare_classes, popular_classes, delete_frame_only,class_in_frame_in_rare_test,class_in_frame_in_rare_train,class_in_frame_in_rare_other)
# delete_frame = frame_can_delete(annotation_path, rare_classes)

Classes only in test: {'w28': 1, 'pclr': 1, 'w56': 1}
Values:  dict_values([1, 1, 1])
Classes only in train: {'w5': 1, 'p20': 2, 'pt': 7, 'p21': 2, 'pcs': 2, 'p24': 2, 'w14': 1}
Values:  dict_values([1, 2, 7, 2, 2, 2, 1])
Classes only in other: {'w48': 1, 'w31': 3, 'w26': 13, 'w50': 3, 'w49': 1, 'w44': 1, 'w1': 1, 'w62': 1, 'p7': 1}
other:  dict_values([1, 3, 13, 3, 1, 1, 1, 1, 1])
Frame có class chỉ có trong test  ['40317', '68066', '65423']
số lượng  3

Frame có class chỉ có trong train  ['50054', '39533', '36401', '36719', '6719', '77771', '57045', '72346', '25439', '30699', '86540', '8216', '9990', '85771', '66927', '75636', '38063']
số lượng  17

Frame có class chỉ có trong other  ['23207', '1201', '74671', '55266', '23473', '48069', '85843', '73073', '14089', '98638', '97512', '84764', '74641', '54175', '13151', '91579', '75772', '98346', '20529', '2364', '98203', '36031', '46372', '69274']
số lượng  24

Frame có class chỉ có trong test và other  []
số lượng  0

Frame có class hi